# 1. Install required libraries. 
(some more may be required for diferent environemnt)

In [ ]:
!pip install llama-index transformers accelerate bitsandbytes 

##2. I changed local as html to text failed due to default ANSI on mac m2 

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
%pip install html2text

## 2. First read the top level page, then we will parse this page to get all links.

In [21]:
from llama_index import SummaryIndex, SimpleWebPageReader
from IPython.display import Markdown, display
import os

master_pages = SimpleWebPageReader().load_data(
    ["https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/match-schedule-fixtures-and-results"
     ]
)

page_text = ""
for master_page in master_pages:
  page_text += master_page.text


In [22]:
def getURL(page):
    """

    :param page: html of web page (here: Python home page) 
    :return: urls in that page 
    """
    start_link = page.find("a href")
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

## 3. For quick testing, only India specific links are saved.

In [25]:
page = page_text
urls = []
demo_filter_text = "india"

while True:
    
    url, n = getURL(page)
    page = page[n:]
    if url:
      if not url.startswith("https://"):
        url = "https://www.espncricinfo.com" + url
        if "icc-cricket-world-cup" in url and demo_filter_text in url:
            urls.append(url)
            #print(url)
    else:
        break
        

In [26]:
print(len(urls))        

33


In [28]:
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)

In [29]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=256, chunk_overlap=20)
node_parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = node_parser.get_nodes_from_documents(documents)
print(f"loaded nodes with {len(nodes)} nodes")

loaded nodes with 1924 nodes


In [7]:
'''

from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.text_splitter import TokenTextSplitter
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
documents = loader.load_data(pages=['2023 Cricket World Cup', '2023 Cricket World Cup statistics', '2023_Cricket_World_Cup_group_stage'], auto_suggest=False)
print(f'Loaded {len(documents)} documents')

# #construct text splitter to split texts into chunks for processing
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=20)

#create node parser to parse nodes from document
node_parser = SimpleNodeParser(text_splitter=text_splitter)

nodes = node_parser.get_nodes_from_documents(documents)
print(f"loaded nodes with {len(nodes)} nodes")

'''

'\n\nfrom llama_index.node_parser import SimpleNodeParser\nfrom llama_index.node_parser.extractors import (\n    MetadataExtractor,\n    SummaryExtractor,\n    QuestionsAnsweredExtractor,\n    TitleExtractor,\n    KeywordExtractor,\n)\nfrom llama_index.text_splitter import TokenTextSplitter\nfrom llama_index import download_loader\n\nWikipediaReader = download_loader("WikipediaReader")\nloader = WikipediaReader()\ndocuments = loader.load_data(pages=[\'2023 Cricket World Cup\', \'2023 Cricket World Cup statistics\', \'2023_Cricket_World_Cup_group_stage\'], auto_suggest=False)\nprint(f\'Loaded {len(documents)} documents\')\n\n# #construct text splitter to split texts into chunks for processing\ntext_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=20)\n\n#create node parser to parse nodes from document\nnode_parser = SimpleNodeParser(text_splitter=text_splitter)\n\nnodes = node_parser.get_nodes_from_documents(documents)\nprint(f"loaded nodes with {len(nodes)} nod

## Use a local text embedding interface. Steps to setup are:

curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

#on m2

cargo install --path router -F candle -F accelerate
model=BAAI/bge-large-en-v1.5
revision=refs/pr/5

text-embeddings-router --model-id $model --revision $revision --port 8080

check git repo for more details:
https://github.com/huggingface/text-embeddings-inference

In [8]:
from llama_index.embeddings import TextEmbeddingsInference

embed_model = TextEmbeddingsInference(
    model_name="BAAI/bge-large-en-v1.5",
    base_url = "http://127.0.0.1:8080",
    #base_url = "http://ec2-##-##-##-##.compute-1.amazonaws.com:8080",
    timeout=60,  # timeout in seconds
    embed_batch_size=10,  # batch size for embedding
)

## Next we download llam2 model quantized to run on CPU.

In [119]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-7B-chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
   # model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4783.16


4562it [14:27,  5.26it/s]                                                                                                                                                                                                                       
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            bl

llm_load_vocab: special tokens definition check successful ( 259/32000 ).
llm_load_print_meta: format           = GGUF V2
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_meta: n_vocab          = 32000
llm_load_print_meta: n_merges         = 0
llm_load_print_meta: n_ctx_train      = 4096
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 32
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_gqa            = 1
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-06
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_meta: n_ff             = 11008
llm_load_print_meta: rope scaling     = linear
llm_load_print_meta: freq_base_train  = 10000.0
llm_load_print_meta: freq_scale_train = 1
llm_load_print_meta:

## and if we have downloaded already, we can load next time. 

In [12]:
#or use a locally downloaded model path once you have downloaded it already. 
#/Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q5_K_M.gguf
#/Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf

import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
model_path = "/Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf"

llm = LlamaCPP(
    model_url=None,
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=model_path,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    #model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llm = LlamaCPP(model_path=model_path)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K 

llama_new_context_with_model: kv self size  = 1950.00 MB
llama_build_graph: non-view tensors processed: 740/740
llama_new_context_with_model: compute buffer total size = 282.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/s/Library/Caches/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tens

llama_new_context_with_model: kv self size  = 1950.00 MB
llama_build_graph: non-view tensors processed: 740/740
llama_new_context_with_model: compute buffer total size = 282.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


## We create a service context

In [121]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512, chunk_overlap=20)

## test 1, use a simple vector store index. This may take some time.

In [31]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex(
    nodes=nodes,
    service_context=service_context,
    show_progress=True
)

Generating embeddings:   0%|                                                                                                                                                                                           | 0/1924 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   1%|▉                                                                                                                                                                                 | 10/1924 [00:03<12:29,  2.55it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   1%|█▊                                                                                                                                                                                | 20/1924 [00:07<11:38,  2.73it/s]

...

Generating embeddings:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 1910/1924 [12:04<00:05,  2.66it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 1920/1924 [12:07<00:01,  2.70it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1924/1924 [12:09<00:00,  2.64it/s]


## first test with query engine. and create a easy test UI with gradio.
Note that if debug=True is enbaled, it will run continuously 

In [39]:
query_engine = index.as_query_engine()

In [38]:
import os
import gradio as gr
def generate(input):
    output = query_engine.query(input)
    return output

demo = gr.Interface(fn=generate, inputs=[gr.Textbox(label="Prompt")], outputs=[gr.Textbox(label="Completion")])
gr.close_all()
demo.launch(share=False, server_port=int(4747), debug=True)

Closing server running on port: 4747
Closing server running on port: 4747
Running on local URL:  http://127.0.0.1:4747

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Llama.generate: prefix-match hit

llama_print_timings:        load time =  136420.82 ms
llama_print_timings:      sample time =      29.97 ms /   134 runs   (    0.22 ms per token,  4470.69 tokens per second)
llama_print_timings: prompt eval time =  257034.65 ms /  1481 tokens (  173.55 ms per token,     5.76 tokens per second)
llama_print_timings:        eval time =  104101.59 ms /   133 runs   (  782.72 ms per token,     1.28 tokens per second)
llama_print_timings:       total time =  361604.44 ms


Keyboard interruption in main thread... closing server.


## Now, we try different options like recurrsive Retriver, saving index in Redis store and neo4j graph

In [35]:
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)

from IPython.display import Markdown, display

In [41]:
for idx, node in enumerate(nodes):
    node.id_ = f"node-{idx}"

In [43]:
query_engine = index.as_query_engine(similarity_top_k=2)

In [44]:
base_retriever = index.as_retriever(similarity_top_k=2)

In [45]:
retrievals = base_retriever.retrieve(
    "Can you tell me about india pakistan match?"
)


INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


In [47]:
from llama_index.response.notebook_utils import display_source_node
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** 587377c1-920a-4879-bc7c-f046bd821733<br>**Similarity:** 0.7316831226917937<br>**Text:** __1:32

Urooj Mumtaz bemoans Pakistan's batting

](/cricket-videos/urooj-mumtaz-bemoans-pakistan-batting-vs-india-1403225)

 _ **India** 192 for 3 (Rohit 86, Shreyas 53*) beat **Pakistan** 191 (Babar
50, Rizwan 49, Bumrah 2-19, Kuldeep 2-35, Jadeja 2-38) by seven wickets _  

The game that always promises and rarely delivers came and went once more,
promising plenty at various stages, and ultimately delivering what a blue mass
of humanity had come to see: a crushing India win. A stunning middle-overs
collapse from Pakistan which had them lose eight wickets for 36 runs turned
what should have been an enthralling run chase into a dull formality.

Each India bowler played a part in their own way, five different players
splitting the ten Pakistan wickets evenly. With Pakistan folding for 191,
India - spearheaded by captain<br>

**Node ID:** 8ccf46e6-a068-4956-8ab8-b5081d0a2560<br>**Similarity:** 0.7135040271385101<br>**Text:** Images

__

__Language

English __

Win Probability

IND 100%

PAKIND

100%50%100%PAK InningsIND Innings

Over 31 • IND 192/3

India won by 7 wickets (with 117 balls remaining)

![](https://wassets.hscicdn.com/static/images/lazyimage-noaspect.svg)Powered
by Smart Stats

Match Coverage

[All Match News __](/series/icc-cricket-world-cup-2023-24-1367856/india-vs-
pakistan-12th-match-1384403/match-news "All Match News")

[

## PCB files complaint with ICC over'inappropriate conduct by spectators' at
India vs Pakistan game

The PCB has also "lodged another formal protest" over delays in visas for
Pakistani journalists and absence of a visa policy for Pakistan fans

![PCB files complaint with ICC over 'inappropriate conduct by spectators' at
India vs Pakistan<br>

In [49]:
from llama_index.query_engine import RetrieverQueryEngine
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

In [51]:
import time

start = time.time()
response = query_engine_base.query(
    "Can you tell me about india pakistan match?"
)
t = time.time() - start
print(str(response))
print("time taken {t}", t)


INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Llama.generate: prefix-match hit


 Of course! The most recent India-Pakistan match was played on March 22nd, 2023, at the World Cup in India. Pakistan batted first and scored 191 runs, with Babar Azam scoring 50 runs and Rizwan scoring 49 runs. In response, India scored 192 for 3, with Rohit Sharma scoring 86 runs and Shreyas Iyer scoring 53 runs. India won the match by seven wickets, with five different Indian bowlers taking five Pakistan wickets evenly. The game was a one-sided affair as Pakistan collapsed in the middle overs, losing eight wickets for 36 runs and ultimately delivering what many had come to see: a crushing India win.
time taken {t} 90.01344418525696



llama_print_timings:        load time =  136420.82 ms
llama_print_timings:      sample time =      29.19 ms /   174 runs   (    0.17 ms per token,  5961.97 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   89054.06 ms /   174 runs   (  511.80 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =   89512.23 ms


In [82]:
# let's make multiple chunk size. But our local embedding does not like any size more than 512 so will keep smaller.
from llama_index.schema import IndexNode
sub_chunk_sizes = [256]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
i = 0
for base_node in nodes:
    for n in sub_node_parsers:
        #print(i)
        i += 1
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)
    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    #print(type(original_node))
    all_nodes.append(original_node)

In [67]:
len(nodes)

1924

In [78]:
all_nodes_dict = {n.node_id: n for n in all_nodes}
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context, show_progress=True
)

Generating embeddings:   0%|                                                                                                                                                                                           | 0/3848 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   0%|▍                                                                                                                                                                                 | 10/3848 [00:03<23:48,  2.69it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   1%|▉                                                                                                                                                                                 | 20/3848 [00:07<22:56,  2.78it/s]

...

Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 3830/3848 [30:15<00:08,  2.23it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 3840/3848 [30:20<00:03,  2.17it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3848/3848 [30:24<00:00,  2.11it/s]


In [87]:
from llama_index.retrievers import RecursiveRetriever
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=5)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [88]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about india pakistan match?"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about india pakistan match?
INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
Retrieved node with id, entering: node-685
Retrieving with query id node-685: Can you tell me about india pakistan match?
Retrieved node with id, entering: node-746
Retrieving with query id node-746: Can you tell me about india pakistan match?
Retrieved node with id, entering: node-642
Retrieving with query id node-642: Can you tell me about india pakistan match?


**Node ID:** node-685<br>**Similarity:** 0.731683132465524<br>**Text:** __1:32

Urooj Mumtaz bemoans Pakistan's batting

](/cricket-videos/urooj-mumtaz-bemoans-pakistan-batting-vs-india-1403225)

 _ **India** 192 for 3 (Rohit 86, Shreyas 53*) beat **Pakistan** 191 (Babar
50, Rizwan 49, Bumrah 2-19, Kuldeep 2-35, Jadeja 2-38) by seven wickets _  

The game that always promises and rarely delivers came and went once more,
promising plenty at various stages, and ultimately delivering what a blue mass
of humanity had come to see: a crushing India win. A stunning middle-overs
collapse from Pakistan which had them lose eight wickets for 36 runs turned
what should have been an enthralling run chase into a dull formality.

Each India bowler played a part in their own way, five different players
splitting the ten Pakistan wickets evenly. With Pakistan folding for 191,
India - spearheaded by captain<br>

**Node ID:** node-746<br>**Similarity:** 0.7135040470884344<br>**Text:** Images

__

__Language

English __

Win Probability

IND 100%

PAKIND

100%50%100%PAK InningsIND Innings

Over 31 • IND 192/3

India won by 7 wickets (with 117 balls remaining)

![](https://wassets.hscicdn.com/static/images/lazyimage-noaspect.svg)Powered
by Smart Stats

Match Coverage

[All Match News __](/series/icc-cricket-world-cup-2023-24-1367856/india-vs-
pakistan-12th-match-1384403/match-news "All Match News")

[

## PCB files complaint with ICC over'inappropriate conduct by spectators' at
India vs Pakistan game

The PCB has also "lodged another formal protest" over delays in visas for
Pakistani journalists and absence of a visa policy for Pakistan fans

![PCB files complaint with ICC over 'inappropriate conduct by spectators' at
India vs Pakistan<br>

**Node ID:** node-642<br>**Similarity:** 0.709095042826496<br>**Text:** Smart Stats

Match Coverage

[All Match News __](/series/icc-cricket-world-cup-2023-24-1367856/india-vs-
pakistan-12th-match-1384403/match-news "All Match News")

[

## PCB files complaint with ICC over'inappropriate conduct by spectators' at
India vs Pakistan game

The PCB has also "lodged another formal protest" over delays in visas for
Pakistani journalists and absence of a visa policy for Pakistan fans

![PCB files complaint with ICC over 'inappropriate conduct by spectators' at
India vs Pakistan game](https://wassets.hscicdn.com/static/images/lazyimage-
noaspect.svg)](/story/world-cup-india-vs-pakistan-pcb-files-complaint-with-
icc-over-inappropriate-conduct-by-spectators-1403871)

[

## ICC chair Barclay on Arthur remark: 'Will review all criticisms'

"We will just take it as it plays out, get to the end<br>

In [89]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [86]:
start = time.time()

response = query_engine_chunk.query(
    "who won the match between India and Pakistan in 2023 world cup?"
)

t = time.time() - start

print(str(response))

Retrieving with query id None: Can you tell me about india pakistan match?
INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
Retrieved node with id, entering: node-685
Retrieving with query id node-685: Can you tell me about india pakistan match?


Llama.generate: prefix-match hit


 Of course! The most recent India vs Pakistan cricket match was played on March 14th, 2022, at the Dubai International Cricket Stadium in Dubai, United Arab Emirates. India won the match by seven wickets, with Rohit Sharma scoring 86 runs and Shreyas Iyer contributing 53 runs to their team's total of 192 for 3. Pakistan struggled in their chase, losing eight wickets for 36 runs and ultimately falling short by seven wickets. The match was a one-day international (ODI) as part of the ICC Men's Cricket World Cup Super League.



llama_print_timings:        load time =  136420.82 ms
llama_print_timings:      sample time =      46.58 ms /   150 runs   (    0.31 ms per token,  3220.40 tokens per second)
llama_print_timings: prompt eval time =   36376.94 ms /    35 tokens ( 1039.34 ms per token,     0.96 tokens per second)
llama_print_timings:        eval time =  176282.15 ms /   149 runs   ( 1183.10 ms per token,     0.85 tokens per second)
llama_print_timings:       total time =  213413.23 ms


In [102]:
from llama_index.vector_stores import RedisVectorStore

In [107]:
vector_store = RedisVectorStore(
    index_name="idx_icc_cwc2023_india",
    index_prefix="sports_",
    redis_url="redis://localhost:6379",
    overwrite=True,
)

In [108]:
from llama_index.storage.storage_context import StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [109]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context, storage_context=storage_context, show_progress=True
)

Generating embeddings:   0%|                                                                                                                                                                                           | 0/3848 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   0%|▍                                                                                                                                                                                 | 10/3848 [00:08<52:10,  1.23it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings:   1%|▉                                                                                                                                                                                 | 20/3848 [00:16<53:13,  1.20it/s]

...

Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 3840/3848 [40:22<00:03,  2.03it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"


Generating embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3848/3848 [40:26<00:00,  1.59it/s]

INFO:llama_index.vector_stores.redis:Creating index idx_icc_cwc2023_india
Creating index idx_icc_cwc2023_india
Creating index idx_icc_cwc2023_india
Creating index idx_icc_cwc2023_india


INFO:llama_index.vector_stores.redis:Added 3848 documents to index idx_icc_cwc2023_india
Added 3848 documents to index idx_icc_cwc2023_india
Added 3848 documents to index idx_icc_cwc2023_india
Added 3848 documents to index idx_icc_cwc2023_india


In [110]:
#persist vector store in redis
vector_store.persist(persist_path="")

INFO:llama_index.vector_stores.redis:Saving index to disk in background
Saving index to disk in background
Saving index to disk in background
Saving index to disk in background


In [122]:
#load vector store from redis
vector_store = RedisVectorStore(
    index_name="idx_icc_cwc2023_india",
    index_prefix="sports_",
    redis_url="redis://localhost:6379",
    overwrite=True,
)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [123]:
import textwrap

query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.query("Can you tell me about india pakistan match in world cup?")
print(textwrap.fill(str(response), 120))

INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8080/embed "HTTP/1.1 200 OK"
INFO:llama_index.vector_stores.redis:Using filters: *
Using filters: *
Using filters: *
Using filters: *
Using filters: *
INFO:llama_index.vector_stores.redis:Querying index idx_icc_cwc2023_india
Querying index idx_icc_cwc2023_india
Querying index idx_icc_cwc2023_india
Querying index idx_icc_cwc2023_india
Querying index idx_icc_cwc2023_india
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['sports_/vector_2dbd5bc5-dd58-417d-9980-a630b985204b', 'sports_/vector_node-684']
Found 2 results for query with id ['sports_/vector_2dbd5bc5-dd58-417d-9980-a630b985204b', 'sports_/vector_node-684']
Found 2 results for query with id ['sports_/vecto


llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =      65.94 ms /   243 runs   (    0.27 ms per token,  3685.22 tokens per second)
llama_print_timings: prompt eval time =  112582.86 ms /   674 tokens (  167.04 ms per token,     5.99 tokens per second)
llama_print_timings:        eval time =  429880.48 ms /   242 runs   ( 1776.37 ms per token,     0.56 tokens per second)
llama_print_timings:       total time =  543767.88 ms


In [118]:
import os
import gradio as gr
def generate(input):
    output = query_engine.query(input)
    return output

demo = gr.Interface(fn=generate, inputs=[gr.Textbox(label="Prompt")], outputs=[gr.Textbox(label="Completion")])
gr.close_all()
demo.launch(share=False, server_port=int(4747), debug=True)

Closing server running on port: 4747
Closing server running on port: 4747
Closing server running on port: 4747
Running on local URL:  http://127.0.0.1:4747

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [131]:
username = "neo4j"
password = "neo4j2023"
url = "bolt://127.0.0.1:7687"
database = "neo4j"

In [135]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import Neo4jGraphStore


graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    service_context=service_context,
)

Llama.generate: prefix-match hit


  Sure! Here are two knowledge triplets that can be extracted from the given text:
1. (World Cup 2023, is, ODI)
2. (AUS, is, playing against, ENG)



llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =       9.89 ms /    51 runs   (    0.19 ms per token,  5154.12 tokens per second)
llama_print_timings: prompt eval time =   66368.98 ms /   463 tokens (  143.35 ms per token,     6.98 tokens per second)
llama_print_timings:        eval time =   79426.93 ms /    50 runs   ( 1588.54 ms per token,     0.63 tokens per second)
llama_print_timings:       total time =  145981.72 ms
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you extract knowledge triplets from the provided text! Here are two triplets I found:
1. (Schedule, is, table)
2. (Videos, is, series)



llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =      10.36 ms /    51 runs   (    0.20 ms per token,  4924.21 tokens per second)
llama_print_timings: prompt eval time =   57082.26 ms /   334 tokens (  170.90 ms per token,     5.85 tokens per second)
llama_print_timings:        eval time =   50843.95 ms /    50 runs   ( 1016.88 ms per token,     0.98 tokens per second)
llama_print_timings:       total time =  108103.66 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =      18.12 ms /    56 runs   (    0.32 ms per token,  3090.51 tokens per second)
llama_print_timings: prompt eval time =   67875.59 ms /   319 tokens (  212.78 ms per token,     4.70 tokens per second)
llama_print_timings:        eval time =  109860.67 ms /    55 runs   ( 1997.47 ms per token,     0.50 tokens per second)
llama_print_timings:       total time =  178100.07 ms
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you extract knowledge triplets from the given text! Here are two triplets I found:
(BH-W, is, 17.2/20 ov)
(BH-W, founded in, 1982)



llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =      19.54 ms /    64 runs   (    0.31 ms per token,  3275.67 tokens per second)
llama_print_timings: prompt eval time =  102729.07 ms /   463 tokens (  221.88 ms per token,     4.51 tokens per second)
llama_print_timings:        eval time =  142896.86 ms /    63 runs   ( 2268.20 ms per token,     0.44 tokens per second)
llama_print_timings:       total time =  246099.52 ms
Llama.generate: prefix-match hit


  I'm happy to help you extract knowledge triplets from the given text! Here are two triplets based on the provided text:
Triplet 1:
(PNJB, is, cricket tournament)
(PNJB, held in, Mirpur)

Triplet 2:
(Punjab, won by, 6 wickets)
(Punjab, won against, Delhi)



llama_print_timings:        load time =   70727.65 ms
llama_print_timings:      sample time =      31.86 ms /    95 runs   (    0.34 ms per token,  2981.51 tokens per second)
llama_print_timings: prompt eval time =   96747.09 ms /   387 tokens (  249.99 ms per token,     4.00 tokens per second)
llama_print_timings:        eval time =  196075.52 ms /    94 runs   ( 2085.91 ms per token,     0.48 tokens per second)
llama_print_timings:       total time =  293459.48 ms
Llama.generate: prefix-match hit


In [ ]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

In [ ]:
response = query_engine.query("Tell me more about india vs pakistan match")